In [1]:
import pandas as pd
import sys
import numpy as np
import pathlib as pl
import zipfile
sys.path.append('../dependencies/')
sys.path.append('../scripts/')
import pyemu
from datetime import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import calendar
from postprocessing import plot_single_real
from post_process_model_output_fn import postprocess

/Users/mnfienen/Documents/GIT/hytest/NHM_CAL/nhm_pestpp/notebooks/../dependencies/pyemu/pst/pst_handler.py:15: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.


In [2]:
curr_model = '01473000'
# curr_model = '05431486'
# curr_model = '09112500'
curr_root = 'prior_mc_reweight'
citer = 0

In [3]:
postproc_nhm = True

if postproc_nhm:
    for cm in ['01473000','05431486','09112500']:
        print(f'postprocessing {cm}')
        c_path = pl.Path(f'../NHM_solutions/nhm_byHWobs_files/{curr_model}')
        postprocess(c_path, '.')

AttributeError: 'Dataset' object has no attribute 'soil_rechr'

In [ ]:
fig_dir = pl.Path(f'../postprocessing/figures/single_reals/{curr_model}')
if not fig_dir.exists():
    fig_dir.mkdir(parents=True)

In [ ]:
res = pd.read_csv(tmp_res_path / f'{curr_root}.0.base.rei', delim_whitespace=True, skiprows=3, index_col=0)


In [ ]:
res = pd.read_csv(tmp_res_path / f'{curr_root}.0.base.rei', delim_whitespace=True, skiprows=3, index_col=0)
res.loc[res.Measured<-999, 'Measured'] = np.nan
res.sample(5)

In [ ]:
res.Group.unique()

In [ ]:
cgroup = 'actet_mean_mon'
[plot_single_real(res, cgroup, citer, curr_root, fig_dir, curr_model) for cgroup in [
    'actet_mean_mon',
    'actet_mon',
    'recharge_ann',
    'soil_moist_mon',
    'soil_moist_ann',
    'runoff_mon',
    'streamflow_daily',
    'sreamflow_mean_mon',
    'streamflow_mon'
]]